In [ ]:
# Importamos librerías externas: 

# Para arrancar el proyecto, necesitamos nuestras herramientas principales, que son las librerías que amplían las capacidades de Python
# Las librerías son herramientas creadas por la comunidad que amplían Python y se instalan con pip install + librería en cuestión

# Permite pedir y enviar información a páginas web y servidores. (Esencial para descargar la API)
import requests

# Organiza y analiza datos en forma de tablas (DataFrames). (Esencial para guardar las películas)
import pandas as pd

# Visualiza todas las columnas de los DataFrames (Muy útil cuando trabajas con DataFrames de Pandas que tienen muchas columnas)
pd.set_option('display.max_columns', None) 

# Conectar y manejar Bases de Datos MySQL. (Necesario para Fases 2 y 3)
import mysql.connector

In [27]:
# FASE 1: EXTRACCIÓN DE DATOS DE PELÍCULAS DESDE LA API

# URL de la API desde donde se obtendrán los datos de películas
url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

print(f"Intentando conectar a la API en: {url}")

# Bloque try/except para manejar posibles errores de conexión
try:
    # Petición GET a la URL para descargar los datos
    respuesta = requests.get(url)

    # Verificamos el código de estado del servidor (200 = OK)
    if respuesta.status_code == 200:
        print("Conexión exitosa. Código de estado:", respuesta.status_code)

        # Convertimos los datos de JSON a estructuras de Python (lista o diccionario)
        datos_pelis = respuesta.json()

        # Visualizamos la estructura de los datos descargados
        print(f"Estructura de los datos descargados en formato JSON: {datos_pelis}")

    else:
        # Si el código de estado no es 200, mostramos mensaje de error
        print(f"Error en la conexión. Código de estado: {respuesta.status_code}. No se pudieron extraer los datos.")

# Captura de errores comunes: falta de conexión a Internet, URL mal escrita o servidor caído
except requests.exceptions.RequestException as e:
    print(f"Ocurrió un error al intentar conectar a la API: {e}")


Intentando conectar a la API en: https://beta.adalab.es/resources/apis/pelis/pelis.json
Conexión exitosa. Código de estado: 200
Estructura de los datos descargados en formato JSON: [{'id': 1, 'titulo': 'The Godfather', 'año': 1972, 'duracion': 175, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 2, 'titulo': 'The Godfather Part II', 'año': 1974, 'duracion': 202, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 3, 'titulo': 'Pulp Fiction', 'año': 1994, 'duracion': 154, 'genero': 'Crimen', 'adultos': True, 'subtitulos': ['es', 'en']}, {'id': 4, 'titulo': 'Forrest Gump', 'año': 1994, 'duracion': 142, 'genero': 'Drama', 'adultos': False, 'subtitulos': ['es', 'en', 'fr']}, {'id': 5, 'titulo': 'The Dark Knight', 'año': 2008, 'duracion': 152, 'genero': 'Acción', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 6, 'titulo': 'Fight Club', 'año': 1999, 'duracion': 139, 'genero': 'Drama', 'adultos': True, 'subtitulos': ['es', 'en']}, {'id': 

In [28]:
# Verificamos el tipo de dato
# Aunque sabemos que es una lista, es buena práctica confirmar el tipo antes de procesarlo
print(f"El tipo de dato es: {type(datos_pelis)}")

El tipo de dato es: <class 'list'>


In [ ]:
# Creamos un DataFrame de Pandas (similar a una tabla de Excel) a partir de la lista de películas
# Un DataFrame organiza los datos en filas y columnas, lo que facilita analizarlos y manipularlos
df_peliculas = pd.DataFrame(datos_pelis)

# Mostramos el DataFrame para comprobar que los datos se han cargado correctamente ( sin índices para mejorar la visualización)
print(df_peliculas.to_string(index=False))

# Mostramos el número total de películas extraídas (deberían ser 100)
print(f"Número total de películas extraídas: {len(df_peliculas)}")

 id                                            titulo  año  duracion          genero  adultos   subtitulos
  1                                     The Godfather 1972       175          Crimen    False     [es, en]
  2                             The Godfather Part II 1974       202          Crimen    False     [es, en]
  3                                      Pulp Fiction 1994       154          Crimen     True     [es, en]
  4                                      Forrest Gump 1994       142           Drama    False [es, en, fr]
  5                                   The Dark Knight 2008       152          Acción    False     [es, en]
  6                                        Fight Club 1999       139           Drama     True     [es, en]
  7                                         Inception 2010       148 Ciencia ficción    False [es, en, de]
  8                                        The Matrix 1999       136 Ciencia ficción    False     [es, en]
  9                          The Shaw

In [ ]:
# La columna subtitulos es la única que contiene una lista [es, en, fr]
# Para poder buscar elementos dentro de esa columna, necesitamos convertir la lista a texto
# Con astype(str), transformamos la lista en una cadena de texto, lo que permite usar búsquedas tipo LIKE
df_peliculas['subtitulos'] = df_peliculas['subtitulos'].astype(str)


In [31]:
# Verificamos la versión de mysql-connector-python
print(f"Versión: {mysql.connector.__version__}")

'''
- Importante para conectar Python con MySQL.
- Versiones muy antiguas pueden dar errores al ejecutar consultas.
- Versiones demasiado nuevas pueden no ser 100% compatibles con tu servidor MySQL.
- Comprueba la versión recomendada en la documentación oficial.
- Si está dentro del rango recomendado, la conexión y consultas deberían funcionar sin problemas.

'''

Versión: 9.5.0


'\n- Importante para conectar Python con MySQL.\n- Versiones muy antiguas pueden dar errores al ejecutar consultas.\n- Versiones demasiado nuevas pueden no ser 100% compatibles con tu servidor MySQL.\n- Comprueba la versión recomendada en la documentación oficial.\n- Si está dentro del rango recomendado, la conexión y consultas deberían funcionar sin problemas.\n\n'

In [ ]:
# FASE 2: CREACIÓN DE LA BASE DE DATOS (VERSIÓN PYTHON)

# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='contraseña',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema


Conexión exitosa


In [33]:
'''
Cambio de la contraseña del usuario root (si es necesario)
Usar una contraseña fuerte, combinando mayúsculas, minúsculas, números y símbolos.
ALTER USER 'root'@'localhost' IDENTIFIED BY 'ContraseñaNueva';
FLUSH PRIVILEGES;
'''

"\nCambio de la contraseña del usuario root (si es necesario)\nUsar una contraseña fuerte, combinando mayúsculas, minúsculas, números y símbolos.\nALTER USER 'root'@'localhost' IDENTIFIED BY 'ContraseñaNueva';\nFLUSH PRIVILEGES;\n"

In [34]:
# B. Creación de la base de datos

try:
    # Creamos un cursor: es el intermediario entre Python y MySQL, necesario para ejecutar consultas SQL
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    # "IF NOT EXISTS" evita errores si la base de datos ya está creada
    query_create_database = "CREATE DATABASE IF NOT EXISTS peliculas_db"

    # Ejecutamos la consulta SQL para crear la base de datos
    mycursor.execute(query_create_database)
    print("Base de datos creada correctamente (o ya existía).")

# Captura errores relacionados con MySQL (por ejemplo, permisos insuficientes)
except mysql.connector.Error as e:  
    print("Ocurrió un error al crear la base de datos:", e)


Base de datos creada correctamente (o ya existía).


In [35]:
# C. Creación de las tablas

try:
    # Seleccionamos la base de datos donde vamos a trabajar
    mycursor.execute("USE peliculas_db")

    # Comando SQL para crear la tabla 'peliculas' si no existe
    query_creacion_tabla = """
    CREATE TABLE IF NOT EXISTS peliculas (
        id INT AUTO_INCREMENT PRIMARY KEY,   # Identificador único
        titulo VARCHAR(255) NOT NULL,        # Título obligatorio
        año YEAR,                            # Año de estreno
        duracion INT,                        # Duración en minutos
        genero VARCHAR(100),                 # Género de la película
        adultos TINYINT(1),                  # 0 = No adultos, 1 = Adultos
        subtitulos VARCHAR(255)              # Idioma de subtítulos
    );
    """

    # Ejecutamos la consulta SQL para crear la tabla
    mycursor.execute(query_creacion_tabla)
    print("Tabla peliculas creada correctamente (o ya existía).")

except mysql.connector.Error as e:
    # Capturamos y mostramos cualquier error de MySQL
    print("Ocurrió un error al crear la tabla:", e)


Tabla peliculas creada correctamente (o ya existía).


In [ ]:
# FASE 3: INSERCIÓN DE DATOS

# Query con %s como placeholders para cada columna
query_insercion = """
INSERT IGNORE INTO peliculas 
(id, titulo, año, duracion, genero, adultos, subtitulos) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
# INSERT IGNORE INTO intenta insertar el registro
# Si es duplicado (por ejemplo, la misma clave primaria o única), simplemente no lo inserta y no genera error
# Si no es duplicado, sí lo inserta normalmente

# Cambiar valores NaN por None para que MySQL los acepte como NULL
df_limpieza = df_peliculas.where(pd.notnull(df_peliculas), None)

# Pasar el DataFrame a lista de listas
datos_peliculas = df_limpieza.values.tolist()

try:
    # Insertar todos los registros
    mycursor.executemany(query_insercion, datos_peliculas)

    # Guardar cambios
    cnx.commit()

    print("Se han insertado", mycursor.rowcount, "registros")

except Exception as e:
    print("Ocurrió un error al insertar los datos:", e)


finally: 
    # Cerrar cursor y conexión
    mycursor.close()

Se han insertado 0 registros


In [ ]:
mycursor = cnx.cursor()

# 1. Películas con duración > 120 minutos
mycursor.execute("SELECT COUNT(id) AS total_peliculas_largas FROM peliculas WHERE duracion > 120")
total_peliculas_largas = mycursor.fetchone()[0]
print("Películas largas (>120 min):", total_peliculas_largas)

# fetchone()[0] cuando solo necesitamos un valor de la fila devuelta


Películas largas (>120 min): 59


In [ ]:
# 2. Películas con subtítulos en español
# Este es un caso especial dentro de las 9 consultas
# porque la columna subtítulos, originalmente, contenía una lista [es, en, fr]
# Para evitar errores al consultar, se convirtió la lista a string usando astype(str)
# Esto permite realizar consultas sobre los subtítulos sin problemas

mycursor.execute("SELECT COUNT(id) AS total_subtitulos_es FROM peliculas WHERE subtitulos LIKE '%es%'")
total_subtitulos_es = mycursor.fetchone()[0]
print("Películas con subtítulos en español:", total_subtitulos_es)

Películas con subtítulos en español: 100


In [41]:
 # 3. Películas con contenido adulto
mycursor.execute("SELECT COUNT(id) AS total_contenido_adulto FROM peliculas WHERE adultos = 1")
total_contenido_adulto = mycursor.fetchone()[0]
print("Películas con contenido adulto:", total_contenido_adulto)

Películas con contenido adulto: 47


In [17]:
 # 4. Película más antigua
mycursor.execute("SELECT titulo, año FROM peliculas ORDER BY año ASC LIMIT 1")
pelicula_mas_antigua = mycursor.fetchone()
print("Película más antigua:", pelicula_mas_antigua)

Película más antigua: ('Citizen Kane', 1941)


In [ ]:
# 5. Promedio de duración por género
mycursor.execute("SELECT genero, AVG(duracion) FROM peliculas GROUP BY genero")
promedios = mycursor.fetchall()

# Para mejorar la visibilidad de los resultados, los mostramos en un DataFrame de Pandas que organiza los datos en columnas
df_promedios = pd.DataFrame(promedios, columns=["Genero", "Duracion_Promedio"])
print(df_promedios)

             Genero Duracion_Promedio
0            Crimen          154.2857
1             Drama          126.2593
2            Acción          139.4444
3   Ciencia ficción          136.3077
4           Romance          139.6667
5            Bélico          161.0000
6          Thriller          121.6667
7           Musical          128.0000
8          Fantasía          159.8000
9          Aventura          133.0000
10        Animación          103.0000
11        Biografía          142.3333
12           Terror          116.5714
13         Suspense          120.0000
14          Comedia          117.0000
15          Western          166.5000


In [ ]:
# 6. Películas por año (Ordena de menor a mayor)
mycursor.execute("SELECT año, COUNT(id) AS total_peliculas FROM peliculas GROUP BY año ORDER BY año ASC")
peliculas_por_año = mycursor.fetchall()

# Para mejorar la visibilidad de los resultados, los mostramos en un DataFrame de Pandas que organiza los datos en columnas
df_peliculas_por_año = pd.DataFrame(peliculas_por_año, columns=["Año", "Total_Peliculas"])
# No mostrar el índice para una mejor presentación ( diferencia con el anterior)
print(df_peliculas_por_año.to_string(index=False))

 Año  Total_Peliculas
1941                1
1942                1
1954                1
1957                1
1958                1
1960                1
1972                1
1973                1
1974                1
1976                2
1977                1
1979                1
1980                3
1982                1
1983                1
1984                1
1985                1
1986                1
1990                1
1991                2
1993                2
1994                4
1995                2
1997                3
1998                3
1999                4
2000                3
2001                5
2002                3
2003                3
2004                3
2006                2
2007                1
2008                4
2009                3
2010                3
2011                1
2012                3
2013                4
2014                3
2015                3
2016                3
2017                4
2018                3
2019      

In [20]:
# 7. Año con más películas
mycursor.execute("SELECT año, COUNT(id) FROM peliculas GROUP BY año ORDER BY COUNT(id) DESC LIMIT 1")
año_mas_peliculas = mycursor.fetchone()
print("Año con más películas:", año_mas_peliculas)

Año con más películas: (2001, 5)


In [ ]:
# 8. Películas por género
mycursor.execute("SELECT genero, COUNT(id) AS total_peliculas FROM peliculas GROUP BY genero")
peliculas_por_genero = mycursor.fetchall()

# Para mejorar la visibilidad de los resultados, los mostramos en un DataFrame de Pandas que organiza los datos en columnas
df_peliculas_por_genero = pd.DataFrame(peliculas_por_genero, columns=["Genero", "Total_Peliculas"])
print(df_peliculas_por_genero.to_string(index=False))

         Genero  Total_Peliculas
         Crimen                7
          Drama               27
         Acción                9
Ciencia ficción               13
        Romance                3
         Bélico                2
       Thriller                6
        Musical                1
       Fantasía                5
       Aventura                3
      Animación                9
      Biografía                3
         Terror                7
       Suspense                2
        Comedia                1
        Western                2


In [ ]:
# 9. Películas que contienen 'Godfather'
mycursor.execute("SELECT titulo FROM peliculas WHERE titulo LIKE '%Godfather%'")
godfather_peliculas = mycursor.fetchall()
print("Películas con 'Godfather' en el título:", godfather_peliculas)

# Cerrar cursor y conexión
mycursor.close()

Películas con 'Godfather' en el título: [('The Godfather',), ('The Godfather Part II',)]
